In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import numpy as np
import pandas as pd
from unidecode import unidecode
from bulletin import default_input, default_output
from bulletin.utils.functions import load_with_progressbar, read_excel, create_occupation_table
from bulletin.utils.static import Municipios
from datetime import datetime, timedelta, date
from os.path import dirname, join, isfile, isdir
from bulletin.systems.casos_confirmados import CasosConfirmados

import pyminizip as pz

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:
def trim_overspace(string):
    parts = filter(lambda part: len(part) > 0, string.split(" "))
    return " ".join(parts)

def normalize_text(txt):
    if not txt is None:
        txt = txt.upper()
        txt = unidecode(txt)
        txt = trim_overspace(txt)
        txt =  "".join([x if ('A' <= x <= 'Z') or (x in [' ','-','_','/','\'']) else '' for x in txt])
    else:
        txt = ''
    
    return txt

In [ ]:
hoje = datetime.today()
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] += '/' + municipios['uf']
# municipios['rs_celepar'] = municipios['rs'].astype('str').str.zfill(2)
# municipios.loc[municipios['uf']=='PR']

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}")

cc.replace('origem')

df_casos = cc.df

df_casos = pd.merge(df_casos.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
df_casos = pd.merge(df_casos.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_atend'})

In [ ]:
# CASOS CONFIRMADOS

casos = df_casos[['ibge_residencia', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'exame', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas']]
casos = casos.rename(columns={'ibge_residencia':'IBGE',
                              'paciente':'Nome',
                              'sexo':'Sexo',
                              'idade':'idade_original',
                              'mun_resid':'Mun_Resid',
                              'mun_atend':'Mun_atend',
                              'rs':'RS',
                              'exame':'Laboratorio',
                              'data_diagnostico':'Dt_diag',
                              'data_comunicacao':'dt_notificacao',
                              'data_1o_sintomas':'dt_inicio_sintomas'})

casos['IBGE'] = casos['IBGE'].astype('int')
casos['idade_original'] = casos['idade_original'].astype('int')

casos.to_csv(join(default_output,'enviar_celepar','CORONAVIRUS.casos_confirmados.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
# ÓBITOS CONFIRMADOS

obitos = df_casos.loc[df_casos['evolucao'] == 2,['ibge_residencia', 'paciente', 'sexo', 'idade', 'mun_resid', 'rs', 'data_cura_obito', 'data_com_evolucao']]
obitos = obitos.rename(columns={'ibge_residencia':'IBGE 6',
                                'paciente':'Nome',
                                'sexo':'Sexo',
                                'idade':'idade_original',
                                'mun_resid':'Mun_Resid',
                                'rs':'RS',
                                'data_cura_obito':'Dt_obito',
                                'data_com_evolucao':'dt_com_obito'})

obitos['IBGE 6'] = obitos['IBGE 6'].astype('int')
obitos['idade_original'] = obitos['idade_original'].astype('int')

obitos.to_csv(join(default_output,'enviar_celepar','CORONAVIRUS.obitos.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
# RECUPERADOS

recuperados = df_casos.loc[df_casos['evolucao'] == 1,['ibge_residencia', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'exame', 'data_1o_sintomas', 'data_comunicacao', 'data_cura_obito', 'data_com_evolucao', 'origem']]

recuperados['Recuperado'] = 'Sim'
recuperados['15º dia de isolamento'] = None
recuperados['Telefone'] = None


recuperados = recuperados.rename(columns={'ibge_residencia':'IBGE 6',
                                          'paciente':'Nome',
                                          'sexo':'Sexo',
                                          'idade':'Idade',
                                          'mun_resid':'Mun Resid',
                                          'mun_atend':'Mun de Atendimento',
                                          'rs':'Regional de Residência',
                                          'exame':'Laboratório',
                                          'data_1o_sintomas':'Início dos sintomas',
                                          'data_comunicacao':'Comunicação',
                                          'data_cura_obito':'Data da atualização',
                                          'data_com_evolucao':'dt_com_recuperado',
                                          'origem':'Fonte'})

recuperados = recuperados[['IBGE 6', 'Nome', 'Sexo', 'Idade', 'Mun Resid', 'Mun de Atendimento', 'Regional de Residência', 'Laboratório', 'Início dos sintomas', 'Comunicação', '15º dia de isolamento', 'Telefone', 'Recuperado', 'Data da atualização', 'Fonte', 'dt_com_recuperado']]


recuperados['Regional de Residência'] = recuperados['Regional de Residência'].astype('int')
recuperados['IBGE 6'] = recuperados['IBGE 6'].astype('int')
recuperados['Idade'] = recuperados['Idade'].astype('int')

recuperados.to_csv(join(default_output,'enviar_celepar','CORONAVIRUS.recuperados.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
# # LEITOS

filename = "ocupacao.xls"
book = read_excel(join(default_input, 'ocupacao_leitos', filename))
# ocupacao = create_occupation_table(book, book.sheet_names()[hoje.day-1])
ocupacao = create_occupation_table(book, book.nsheets-1)
ocupacao.to_csv(join(default_output,'enviar_celepar', 'CORONAVIRUS.leitos.csv'), index = False, sep=';', encoding='utf-8-sig')

In [ ]:
# TESTES

testes = pd.DataFrame([[10600, 74200, 427980, pd.read_csv(join(default_input, 'queries', 'testes_rapidos_diario.csv')).iloc[0,0]]], columns=['EXAMES DO MÉTODO RT-PCR (por dia)', 'EXAMES DO MÉTODO RT-PCR (por semana)', 'One Step Test', 'Testes de Antígenos (realizados nos últimos 7 dias)'])
testes.to_csv(join(default_output,'enviar_celepar','CORONAVIRUS.testes.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
# ARQUIVO COMPLETO ENVIADO

cc_vacinados = CasosConfirmados('cc_vacinados')
cc_vacinados.load()

csv_geral_new = cc_vacinados.comunicacao_csvGeralNew()
csv_geral_new.to_csv(join(default_output, 'enviar_celepar',  f"CORONAVIRUS.INFORME_EPIDEMIOLOGICO_COMPLETO.csv"), index=False, sep=';',encoding='utf-8-sig')

In [ ]:
from os import remove, listdir, chdir

# GERA ARQUIVOS COMPACTADOS
dirpath = join(default_output, 'enviar_celepar')
archives = [ f for f in listdir(dirpath) if isfile(join(dirpath, f)) ]

chdir(dirpath)

pz.compress_multiple(archives, [], join(dirpath, f"COVID-19_celepar.zip"), None, 9)

chdir(current_dir)


# # REMOVE ARQUIVOS DE HOJE
# for i in archives:
#     try:
#         remove(join(dirpath, i))
#     except:
#         pass